### Uncomment and run the following cells if you work on Google Colab :) Don't forget to change your runtime type to GPU!

In [1]:
!git clone https://github.com/kstathou/vector_engine

Cloning into 'vector_engine'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 74 (delta 32), reused 59 (delta 18), pack-reused 0
Unpacking objects: 100% (74/74), done.


In [2]:
cd vector_engine

/content/vector_engine


In [3]:
pip install -r requirements.txt

Obtaining file:///content/vector_engine (from -r requirements.txt (line 9))
     |████████████████████████████████| 804.1MB 24kB/s 
     |████████████████████████████████| 1.1MB 29.9MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 
     |████████████████████████████████| 10.5MB 73kB/s 
     |████████████████████████████████| 7.1MB 32.2MB/s 
     |████████████████████████████████| 14.5MB 229kB/s 
     |████████████████████████████████| 71kB 9.2MB/s 
     |████████████████████████████████| 7.1MB 16.1MB/s 
     |████████████████████████████████| 3.0MB 32.6MB/s 
     |████████████████████████████████| 901kB 31.5MB/s 
     |████████████████████████████████| 1.2MB 29.8MB/s 
     |████████████████████████████████| 133kB 41.8MB/s 
     |████████████████████████████████| 7.7MB 33.8MB/s 
     |████████████████████████████████| 4.2MB 29.8MB/s 
     |████████████████████████████████| 112kB 37.9MB/s 
     |████████████████████████████████| 81kB 10.2MB/s 
     |████████████████████████████

In [4]:
!apt-get install mecab mecab-ipadic-utf8 python-mecab libmecab-dev
!pip install mecab-python3 fugashi ipadic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libmecab2 mecab-ipadic mecab-jumandic mecab-jumandic-utf8 mecab-utils
The following NEW packages will be installed:
  libmecab-dev libmecab2 mecab mecab-ipadic mecab-ipadic-utf8 mecab-jumandic
  mecab-jumandic-utf8 mecab-utils python-mecab
0 upgraded, 9 newly installed, 0 to remove and 39 not upgraded.
Need to get 29.0 MB of archives.
After this operation, 277 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab2 amd64 0.996-5 [257 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab-dev amd64 0.996-5 [308 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-utils amd64 0.996-5 [4,856 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-jumandic-utf8 all 7.0-20130310-4 [16.2 MB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/

### Let's begin!

In [5]:
%load_ext autoreload

In [6]:
%autoreload 2
# Used to import data from local.
import pandas as pd

# Used to create the dense document vectors.
import torch
from sentence_transformers import SentenceTransformer

# Used to create and store the Faiss index.
import faiss
import numpy as np
import pickle
from pathlib import Path

# Used to do vector searches and display the results.
from vector_engine.utils import vector_search, id2details

Stored and processed data in s3

In [7]:
# Read a CSV in a table
# df = pd.read_csv('data/misinformation_papers.csv')

In [8]:
# 出典: https://qiita.com/sonoisa/items/775ac4c7871ced6ed4c3 で公開されている「いらすとや」さんの画像タイトル抜粋（「のイラスト」「のマーク」「のキャラクター」という文言を削った）
sentences = ["お辞儀をしている男性会社員", "笑い袋", "テクニカルエバンジェリスト（女性）", "戦うAI", "笑う男性（5段階）", 
"漫才師", "お辞儀をしている医者（女性）", "お辞儀をしている薬剤師", "福笑いをしている人", "AIの家族", "コント師", 
"福笑い（女性）", "お辞儀をしている犬", "苦笑いをする女性", "お辞儀をしている医者", "いろいろな漫符", 
"雛人形「仕丁・三人上戸」", "ダンス「踊る男性」", "拍手をしている人", "定年（男性）", "ものまね芸人", "福笑いのおたふく", 
"お辞儀をしている看護師（男性）", "愛想笑い", "福笑い（ひょっとこ）", "成長する人工知能", "苦笑いをする男性", 
"運動会「徒競走・白組」", "人工知能と喧嘩をする人", "人工知能", "ありがた迷惑", "お辞儀をしているクマ", "笑う女性（5段階）", 
"人工知能とメールをする人（男性）", "技術書", "笑いをこらえる人（女性）", "ダンス「踊る女性」", "お辞儀をしている猫", 
"福笑い（男性）", "武器を持つAI", "作曲する人工知能", "縄跳びを飛んでいる女性", "福笑い（おかめ）", "茅の輪くぐり", "表情", 
"AIと仲良くなる人間", "お笑い芸人「漫才師」", "人工知能とメールをする人（女性）", "人工知能と戦う囲碁の棋士", "拍手している女の子", 
"検索する人工知能", "ピースサインを出す人（女性）", "啓示を受けた人（女性）", "仕事をする人工知能", "一輪車に乗る女の子", 
"お辞儀をしているウサギ", "走る猫（笑顔）", "人工知能と戦う将棋の棋士", "遠足「お弁当・男の子・女の子」", "心を持ったAI", 
"プレゼントをもらって喜ぶ女の子", "技術書（バラバラ）", "いろいろな表情の酔っぱらい（男性）", "拍手している人（棒人間）", 
"仕事を奪う人工知能", "文章を書く人工知能", "いろいろな映画の「つづく」", "絵を描く人工知能", "拍手している男の子", "ハリセン", 
"人工知能と仲良くする人たち", "ON AIRランプ", "いろいろな表情の酔っぱらい（女性）", "徹夜明けの笑顔（女性）", 
"徹夜明けの笑顔（男性）", "お辞儀をしている女性会社員", "バンザイをしているお婆さん", "画像認識をするAI", 
"芸人の男の子（将来の夢）", "料理「女性」", "ピコピコハンマー", "鏡を見る人（笑顔の男性）", "笑いをこらえる人（男性）", 
"シンギュラリティ", "人工知能に仕事を任せる人", "スマートスピーカー", "学ぶ人工知能", "人工知能・AI", "英語のアルファベット", 
"お金を見つめてニヤけている男性", "「ありがとう」と言っている人", "定年（女性）", "テクニカルエバンジェリスト（男性）", 
"スタンディングオベーション"]

In [9]:
df = pd.DataFrame(data=sentences, columns=['text'])
df["id"] = df.index

In [10]:
df.head(3)

,text,id
0,お辞儀をしている男性会社員,0
1,笑い袋,1
2,テクニカルエバンジェリスト（女性）,2


In [11]:
# print(f"Misinformation, disinformation and fake news papers: {df.id.unique().shape[0]}")
print(f"texts: {df.id.unique().shape[0]}")

texts: 94


The [Sentence Transformers library](https://github.com/UKPLab/sentence-transformers) offers pretrained transformers that produce SOTA sentence embeddings. Checkout this [spreadsheet](https://docs.google.com/spreadsheets/d/14QplCdTCDwEmTqrn1LH4yrbKvdogK4oQvYO1K1aPR5M/) with all the available models.

In this tutorial, we will use the `distilbert-base-nli-stsb-mean-tokens` model which has the best performance on Semantic Textual Similarity tasks among the DistilBERT versions. Moreover, although it's slightly worse than BERT, it is quite faster thanks to having a smaller size.

I use the same model in [Orion's semantic search engine](https://www.orion-search.org/)!

In [12]:
# Instantiate the sentence-level DistilBERT
# model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
# model = SentenceTransformer('cl-tohoku/bert-base-japanese-whole-word-masking')
from sentence_transformers import models
transformer = models.BERT('cl-tohoku/bert-base-japanese-whole-word-masking')
pooling = models.Pooling(transformer.get_word_embedding_dimension(), pooling_mode_mean_tokens=True, pooling_mode_cls_token=False, pooling_mode_max_tokens=False)
model = SentenceTransformer(modules=[transformer, pooling])

# Check if GPU is available and use it
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
print(model.device)


cuda:0


In [13]:
# Convert abstracts to vectors
# embeddings = model.encode(df.abstract.to_list(), show_progress_bar=True)
embeddings = model.encode(df.text.to_list(), show_progress_bar=True)

In [14]:
# print(f'Shape of the vectorised abstract: {embeddings[0].shape}')
print(f'Shape of the vectorised text: {embeddings[0].shape}')

Shape of the vectorised text: (768,)


## Vector similarity search with Faiss
[Faiss](https://github.com/facebookresearch/faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, even ones that do not fit in RAM. 
    
Faiss is built around the `Index` object which contains, and sometimes preprocesses, the searchable vectors. Faiss has a large collection of [indexes](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes). You can even create [composite indexes](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes-(composite)). Faiss handles collections of vectors of a fixed dimensionality d, typically a few 10s to 100s.

**Note**: Faiss uses only 32-bit floating point matrices. This means that you will have to change the data type of the input before building the index.

To learn more about Faiss, you can read their paper on [arXiv](https://arxiv.org/abs/1702.08734).

Here, we will the `IndexFlatL2` index:
- It's a simple index that performs a brute-force L2 distance search
- It scales linearly. It will work fine with our data but you might want to try [faster indexes](https://github.com/facebookresearch/faiss/wiki/Faster-search) if you work will millions of vectors.

To create an index with the `misinformation` abstract vectors, we will:
1. Change the data type of the abstract vectors to float32.
2. Build an index and pass it the dimension of the vectors it will operate on.
3. Pass the index to IndexIDMap, an object that enables us to provide a custom list of IDs for the indexed vectors.
4. Add the abstract vectors and their ID mapping to the index. In our case, we will map vectors to their paper IDs from MAG.

In [15]:
# Step 1: Change data type
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

# Step 2: Instantiate the index
index = faiss.IndexFlatL2(embeddings.shape[1])

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(embeddings, df.id.values)

print(f"Number of vectors in the Faiss index: {index.ntotal}")

Number of vectors in the Faiss index: 94


### Searching the index
The index we built will perform a k-nearest-neighbour search. We have to provide the number of neighbours to be returned. 

Let's query the index with an abstract from our dataset and retrieve the 10 most relevant documents. **The first one must be our query!**


In [16]:
# Paper abstract
#df.iloc[5415, 1]
df.iloc[0, 0]

'お辞儀をしている男性会社員'

In [17]:
# Retrieve the 10 nearest neighbours
# D, I = index.search(np.array([embeddings[5415]]), k=10)
D, I = index.search(np.array([embeddings[0]]), k=10)
# print(f'L2 distance: {D.flatten().tolist()}\n\nMAG paper IDs: {I.flatten().tolist()}')
print(f'L2 distance: {D.flatten().tolist()}\n\nIDs: {I.flatten().tolist()}')

L2 distance: [0.0, 1.0819151401519775, 13.152276039123535, 13.542501449584961, 16.09006690979004, 16.88435935974121, 17.675594329833984, 18.09502601623535, 20.068321228027344, 23.48537826538086]

IDs: [0, 75, 22, 6, 14, 37, 55, 31, 12, 7]


In [18]:
# Fetch the paper titles based on their index
# id2details(df, I, 'original_title')
id2details(df, I, 'text')

[['お辞儀をしている男性会社員'],
 ['お辞儀をしている女性会社員'],
 ['お辞儀をしている看護師（男性）'],
 ['お辞儀をしている医者（女性）'],
 ['お辞儀をしている医者'],
 ['お辞儀をしている猫'],
 ['お辞儀をしているウサギ'],
 ['お辞儀をしているクマ'],
 ['お辞儀をしている犬'],
 ['お辞儀をしている薬剤師']]

In [19]:
# Fetch the paper abstracts based on their index
#id2details(df, I, 'abstract')
id2details(df, I, 'text')

[['お辞儀をしている男性会社員'],
 ['お辞儀をしている女性会社員'],
 ['お辞儀をしている看護師（男性）'],
 ['お辞儀をしている医者（女性）'],
 ['お辞儀をしている医者'],
 ['お辞儀をしている猫'],
 ['お辞儀をしているウサギ'],
 ['お辞儀をしているクマ'],
 ['お辞儀をしている犬'],
 ['お辞儀をしている薬剤師']]


## Putting all together

So far, we've built a Faiss index using the misinformation abstract vectors we encoded with a sentence-DistilBERT model. That's helpful but in a real case scenario, we would have to work with unseen data. To query the index with an unseen query and retrieve its most relevant documents, we would have to do the following:

1. Encode the query with the same sentence-DistilBERT model we used for the rest of the abstract vectors.
2. Change its data type to float32.
3. Search the index with the encoded query.

Here, we will use the introduction of an article published on [HKS Misinformation Review](https://misinforeview.hks.harvard.edu/article/can-whatsapp-benefit-from-debunked-fact-checked-stories-to-reduce-misinformation/).


In [20]:
user_query = """
WhatsApp was alleged to have been widely used to spread misinformation and propaganda 
during the 2018 elections in Brazil and the 2019 elections in India. Due to the 
private encrypted nature of the messages on WhatsApp, it is hard to track the dissemination 
of misinformation at scale. In this work, using public WhatsApp data from Brazil and India, we 
observe that misinformation has been largely shared on WhatsApp public groups even after they 
were already fact-checked by popular fact-checking agencies. This represents a significant portion 
of misinformation spread in both Brazil and India in the groups analyzed. We posit that such 
misinformation content could be prevented if WhatsApp had a means to flag already fact-checked 
content. To this end, we propose an architecture that could be implemented by WhatsApp to counter 
such misinformation. Our proposal respects the current end-to-end encryption architecture on WhatsApp, 
thus protecting users’ privacy while providing an approach to detect the misinformation that benefits 
from fact-checking efforts.
"""

In [21]:
user_query = """
暴走した人工知能
"""

In [22]:
# For convenience, I've wrapped all steps in the vector_search function.
# It takes four arguments: 
# A query, the sentence-level transformer, the Faiss index and the number of requested results
D, I = vector_search([user_query], model, index, num_results=10)
#print(f'L2 distance: {D.flatten().tolist()}\n\nMAG paper IDs: {I.flatten().tolist()}')
print(f'L2 distance: {D.flatten().tolist()}\n\nIDs: {I.flatten().tolist()}')

L2 distance: [29.917055130004883, 31.927461624145508, 33.03748321533203, 34.51005554199219, 34.71612548828125, 35.10759735107422, 37.14727783203125, 37.172996520996094, 37.47172927856445, 39.13682174682617]

IDs: [25, 70, 64, 57, 48, 28, 33, 40, 47, 50]


In [23]:
# Fetching the paper titles based on their index
# id2details(df, I, 'original_title')
id2details(df, I, 'text')

[['成長する人工知能'],
 ['人工知能と仲良くする人たち'],
 ['仕事を奪う人工知能'],
 ['人工知能と戦う将棋の棋士'],
 ['人工知能と戦う囲碁の棋士'],
 ['人工知能と喧嘩をする人'],
 ['人工知能とメールをする人（男性）'],
 ['作曲する人工知能'],
 ['人工知能とメールをする人（女性）'],
 ['検索する人工知能']]

In [24]:
# Define project base directory
# Change the index from 1 to 0 if you run this on Google Colab
# project_dir = Path('notebooks').resolve().parents[1]
project_dir = Path('notebooks').resolve().parents[0]
print(project_dir)

# Serialise index and store it as a pickle
with open(f"{project_dir}/models/faiss_index.pickle", "wb") as h:
    pickle.dump(faiss.serialize_index(index), h)

/content/vector_engine


## Provide your credentials to the runtime

In [25]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Optional: Enable data table display

In [26]:
%load_ext google.colab.data_table

In [27]:
#!pip install --upgrade google-cloud-bigquery[pandas,pyarrow]

In [28]:
df.head(10)

,text,id
0,お辞儀をしている男性会社員,0
1,笑い袋,1
2,テクニカルエバンジェリスト（女性）,2
3,戦うAI,3
4,笑う男性（5段階）,4
5,漫才師,5
6,お辞儀をしている医者（女性）,6
7,お辞儀をしている薬剤師,7
8,福笑いをしている人,8
9,AIの家族,9


In [29]:
ebd = np.array([embedding for embedding in embeddings]).astype("float32").tolist()

#for embedding in embeddings:
#  np.array([embedding]).astype("float32")

df["embedding"] = ebd


In [30]:
df['embedding'] = df['embedding'].astype(str)

In [31]:
df.head()

,text,id,embedding
0,お辞儀をしている男性会社員,0,"[-0.1495862454175949, -0.2155923992395401, -0...."
1,笑い袋,1,"[-0.11451064050197601, -0.06181585416197777, -..."
2,テクニカルエバンジェリスト（女性）,2,"[-0.17782121896743774, 0.36356180906295776, -0..."
3,戦うAI,3,"[-0.524259090423584, 0.30210334062576294, 0.13..."
4,笑う男性（5段階）,4,"[-0.2320312112569809, 0.0705299824476242, 0.08..."


In [32]:
#project_id = 'bwing-230309'
project_id = 'learnlearn-208609'

In [33]:
import datetime

from google.cloud import bigquery
import pandas
import pytz

# Construct a BigQuery client object.
#client = bigquery.Client()
client = bigquery.Client(project=project_id)

# TODO(developer): Set table_id to the ID of the table to create.
# table_id = "your-project.your_dataset.your_table_name"
table_id = "learnlearn-208609.test.embedding"

job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        # Specify the type of columns whose type cannot be auto-detected. For
        # example the "title" column uses pandas dtype "object", so its
        # data type is ambiguous.
        #bigquery.SchemaField("title", bigquery.enums.SqlTypeNames.STRING),
        # Indexes are written if included in the schema by name.
        #bigquery.SchemaField("wikidata_id", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("id", bigquery.enums.SqlTypeNames.INTEGER),
        bigquery.SchemaField("text", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("embedding", bigquery.enums.SqlTypeNames.STRING),
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)

job = client.load_table_from_dataframe(
    #dataframe, table_id, job_config=job_config
    df, table_id, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 94 rows and 3 columns to learnlearn-208609.test.embedding


In [34]:
from google.cloud import bigquery

def query(project_id, text):
  client = bigquery.Client(project=project_id)

  print((project_id, text))
  
  df = client.query(f'''
CREATE TEMP FUNCTION
  JSON_EXTRACT_ARRAY_TO_FLOAT(input STRING)
  RETURNS ARRAY<FLOAT64>
  LANGUAGE js AS """  
return JSON.parse(input);
""";
WITH t AS (
SELECT
  text,
  JSON_EXTRACT_ARRAY_TO_FLOAT(embedding) AS embedding
FROM
  `learnlearn-208609.test.embedding`
)
SELECT
  c.k1 as input_text_title,
  c.k2 as similar_text_title,
  SUM(vv1*vv2) / (SQRT(SUM(POW(vv1,2))) * SQRT(SUM(POW(vv2,2)))) AS similarity
FROM
(
  SELECT
    a.key k1, a.val v1, b.key k2, b.val v2
  FROM
  (
    SELECT text key, embedding val
    FROM t
    WHERE text LIKE '{text}'
    LIMIT 1
   ) a
  CROSS JOIN
  (
    SELECT text key, embedding val
    FROM t
  ) b
) c
, UNNEST(c.v1) vv1 with offset ind1 JOIN UNNEST(c.v2) vv2 with offset ind2 ON (ind1=ind2)
GROUP BY c.k1, c.k2
ORDER BY similarity DESC
LIMIT 10
  ''').to_dataframe()
  # print(df.head(10))
  return df

In [35]:
text = "成長する人工知能"

df_result = query(project_id, text)

('learnlearn-208609', '成長する人工知能')


In [36]:
df_result

,input_text_title,similar_text_title,similarity
0,成長する人工知能,成長する人工知能,1.000000
1,成長する人工知能,検索する人工知能,0.931909
2,成長する人工知能,作曲する人工知能,0.917097
3,成長する人工知能,仕事をする人工知能,0.910704
4,成長する人工知能,人工知能,0.908365
5,成長する人工知能,学ぶ人工知能,0.895540
6,成長する人工知能,文章を書く人工知能,0.894384
7,成長する人工知能,絵を描く人工知能,0.889271
8,成長する人工知能,人工知能と仲良くする人たち,0.887614
9,成長する人工知能,仕事を奪う人工知能,0.886524
